In [1]:
%matplotlib notebook
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog

from astropy.io.fits import getheader
from astropy import units as u
from astropy.coordinates import SkyCoord

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [2]:
def load_detcats(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_vtp.detect'):
            data_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            if len(data_part) > 0:
                data_part['Field'] = field
                try:
                    hdr = getheader(f'{outpath}/{field}/{field}_vtp_high.detect', ext=1)
                except FileNotFoundError:
                    continue
                data_part['Field_EXP'] = hdr['EXPOSURE']
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data


def load_MCMCfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_mcmcfits.txt'):
            try:
                fit_part = Table.read(f'{outpath}/{field}/{field}_mcmcfits.txt', format='ascii', header_start=0)
            except :
                print(field)
                raise (FileNotFoundError)
            src_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            
            fit_part.rename_column('ID', 'INDEX')
            try:
                data_part = join(fit_part, src_part, keys='INDEX', join_type='inner')
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

In [3]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

datapath = '../data_full'
data.replace(-1, np.nan, inplace=True)

fields = [n.replace(' ', '_') for n in data['NAME']]
fits = load_MCMCfits(datapath, fields)

cats = load_detcats(datapath, fields)

# check to see which fields have XRT data
field_mask = [True if os.path.isdir(f'../data_full/{n.replace(" ", "_")}') else False for n in data['NAME']]
                                    
data['SWIFT'] = False
data['SWIFT'] = field_mask

In [4]:
data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC']][data['SWIFT'] == True][:10]

,NAME,INDEX_PSZ2,RA,DEC
0,PSZ2 G000.04+45.13,1.0,229.190512,-1.017222
2,PSZ2 G000.77-35.69,4.0,307.972844,-40.598725
10,PSZ2 G003.91-42.03,12.0,316.467508,-38.753160
16,PSZ2 G006.49+50.56,21.0,227.728950,5.768101
17,PSZ2 G006.68-35.55,22.0,308.705933,-35.825081
18,PSZ2 G006.76+30.45,23.0,243.954880,-6.152363
19,PSZ2 G006.82-34.71,24.0,307.710992,-35.564128
21,PSZ2 G007.76-36.83,28.0,310.439291,-35.187758
24,PSZ2 G008.80-35.18,32.0,308.659507,-34.061514
42,PSZ1 G018.09+26.07,NaN,253.121758,-0.247058


In [5]:
# add more columns
data['XRT_exp'] = np.nan
data['N_H'] = np.nan
data['N_net'] = np.nan
data['SNR'] = np.nan
data['FLUX'] = np.nan

In [45]:
data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC']][data['SWIFT'] == True][:10]

,NAME,INDEX_PSZ2,RA,DEC
0,PSZ2 G000.04+45.13,1.0,229.190512,-1.017222
2,PSZ2 G000.77-35.69,4.0,307.972844,-40.598725
10,PSZ2 G003.91-42.03,12.0,316.467508,-38.753160
16,PSZ2 G006.49+50.56,21.0,227.728950,5.768101
17,PSZ2 G006.68-35.55,22.0,308.705933,-35.825081
18,PSZ2 G006.76+30.45,23.0,243.954880,-6.152363
19,PSZ2 G006.82-34.71,24.0,307.710992,-35.564128
21,PSZ2 G007.76-36.83,28.0,310.439291,-35.187758
24,PSZ2 G008.80-35.18,32.0,308.659507,-34.061514
42,PSZ1 G018.09+26.07,NaN,253.121758,-0.247058


In [11]:
cats.remove_column('R')

In [27]:
df_cats = cats.to_pandas()

In [28]:
df_cats.Field = df_cats.Field.str.replace('_', ' ')

In [29]:
df_cats

,INDEX,RA_VTP,RA_ERR_VTP,DEC_VTP,DEC_ERR_VTP,X_VTP,Y_VTP,X_ERR_VTP,Y_ERR_VTP,SRC_AREA,...,Y,X_ERR,Y_ERR,RA,DEC,RA_ERR,DEC_ERR,Extended,Field,Field_EXP
0,1,229.335259,0.001774,-0.721653,0.001213,258.678144,743.394988,2.708676,1.852612,1817.846558,...,740.673469,6.695025,5.258407,229.337053,-0.723435,0.004389,0.003447,1,PSZ2 G000.04+45.13,8329.250600
1,2,229.186521,0.000726,-0.971285,0.000650,485.808548,362.169757,1.108935,0.993251,6320.484375,...,362.456233,15.425047,14.295588,229.186290,-0.971098,0.010112,0.009372,1,PSZ2 G000.04+45.13,8329.250600
2,1,307.965431,0.001426,-40.622707,0.000688,506.375934,520.052659,1.653208,1.051192,4761.302246,...,519.012048,7.279986,4.999410,307.966691,-40.623388,0.004772,0.003277,1,PSZ2 G000.77-35.69,1341.405500
3,1,316.480235,0.004867,-38.756891,0.002266,588.941519,520.483058,5.799269,3.455429,7955.410645,...,518.131868,11.264661,7.728498,316.479806,-38.758431,0.007385,0.005066,1,PSZ2 G003.91-42.03,1213.533200
4,1,227.776800,0.000070,5.689837,0.000070,433.939792,360.105337,0.106581,0.106658,92.791664,...,360.105769,13.556362,13.665987,227.776741,5.689837,0.008887,0.008959,1,PSZ2 G006.49+50.56,34425.229003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,1,294.042797,0.001028,-45.315792,0.000656,587.596725,462.090073,1.105138,1.000272,4649.674805,...,462.970803,8.566162,8.351214,294.043945,-45.315217,0.005616,0.005475,1,PSZ2 G353.36-26.49,3457.566700
1352,1,219.594268,0.000987,3.663676,0.000951,522.647043,417.901123,1.503624,1.452769,4737.895020,...,419.313869,9.974069,10.060678,219.594966,3.664602,0.006539,0.006595,1,PSZ2 G355.07+55.32,5147.486900
1353,1,326.743521,0.002784,-43.897442,0.001262,424.876650,616.789416,3.061995,1.929587,7471.530273,...,617.780952,8.676639,6.171915,326.743988,-43.896793,0.005688,0.004046,1,PSZ2 G356.04-49.50,1002.920000
1354,1,317.816268,0.003165,-44.410491,0.002118,511.334881,472.932581,3.453743,3.234653,5576.195801,...,477.098039,6.632127,6.573672,317.813340,-44.407763,0.004348,0.004309,1,PSZ2 G356.21-43.11,2427.066400


In [30]:
df_data = data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'XRT_exp', 'N_H', 'N_net', 'SNR', 'FLUX']][data['SWIFT']]

In [64]:
df_m = df_data.merge(df_cats,                                                      
               how='inner',                                              
               left_on='NAME',                                          
               right_on='Field',                                           
               suffixes=('_data', '_detc')) 

In [65]:
df_m.columns

Index(['NAME', 'INDEX_PSZ2', 'RA_data', 'DEC_data', 'XRT_exp', 'N_H', 'N_net',
       'SNR', 'FLUX', 'INDEX', 'RA_VTP', 'RA_ERR_VTP', 'DEC_VTP',
       'DEC_ERR_VTP', 'X_VTP', 'Y_VTP', 'X_ERR_VTP', 'Y_ERR_VTP', 'SRC_AREA',
       'NET_COUNTS', 'NET_COUNTS_ERR', 'BKG_COUNTS', 'BKG_COUNTS_ERR',
       'NET_RATE', 'NET_RATE_ERR', 'BKG_RATE', 'BKG_RATE_ERR', 'EXPTIME',
       'SRC_CUTOFF', 'FSP', 'EDGE_OF_FIELD', 'SHAPE', 'ROTANG', 'COMPONENT',
       'HIGH', 'X', 'Y', 'X_ERR', 'Y_ERR', 'RA_detc', 'DEC_detc', 'RA_ERR',
       'DEC_ERR', 'Extended', 'Field', 'Field_EXP'],
      dtype='object')

In [66]:
sep = np.ones(len(df_m))
for index, row in df_m.iterrows():
    psz_position = SkyCoord(row['RA_data'], row['DEC_data'], frame='icrs', unit=u.deg)
    src_position = SkyCoord(row['RA_detc'], row['DEC_detc'], frame='icrs', unit=u.deg)
    sep[index] = psz_position.separation(src_position).arcmin

In [67]:
sep

array([19.69789376,  2.77902045,  1.50606948, ...,  2.12950298,
        1.82836717,  3.166264  ])

In [68]:
df_m['Sep'] = sep

In [69]:
df_close_ext = df_m[(df_m.Sep < 6) & (df_m.Extended == 1)]

In [73]:
df_close_ext[['NAME', 'RA_detc', 'DEC_detc', 'INDEX', 'Sep', 'EXPTIME']]

,NAME,RA_detc,DEC_detc,INDEX,Sep,EXPTIME
1,PSZ2 G000.04+45.13,229.186290,-0.971098,2,2.779020,7941.609863
2,PSZ2 G000.77-35.69,307.966691,-40.623388,1,1.506069,1267.050293
3,PSZ2 G003.91-42.03,316.479806,-38.758431,1,0.656611,1121.546387
4,PSZ2 G006.49+50.56,227.776741,5.689837,1,5.494694,40198.839844
5,PSZ2 G006.49+50.56,227.752198,5.821058,2,3.467228,39579.894531
...,...,...,...,...,...,...
1351,PSZ2 G353.36-26.49,294.043945,-45.315217,1,2.728880,3244.882080
1352,PSZ2 G355.07+55.32,219.594966,3.664602,1,1.502377,4587.205566
1353,PSZ2 G356.04-49.50,326.743988,-43.896793,1,2.129503,922.871826
1354,PSZ2 G356.21-43.11,317.813340,-44.407763,1,1.828367,2350.517334


In [74]:
mask = df_close_ext[['NAME', 'RA_detc']].duplicated('NAME', keep='first')

In [80]:
df_close_ext2 = df_close_ext.copy()

In [81]:
df_close_ext2.NAME[mask] = ''

/home/boada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'INDEX', 'Sep', 'EXPTIME']][:25]

,NAME,RA_detc,DEC_detc,INDEX,Sep,EXPTIME
1,PSZ2 G000.04+45.13,229.186290,-0.971098,2,2.779020,7941.609863
2,PSZ2 G000.77-35.69,307.966691,-40.623388,1,1.506069,1267.050293
3,PSZ2 G003.91-42.03,316.479806,-38.758431,1,0.656611,1121.546387
4,PSZ2 G006.49+50.56,227.776741,5.689837,1,5.494694,40198.839844
5,,227.752198,5.821058,2,3.467228,39579.894531
6,,227.734097,5.687358,3,4.854370,39736.207031
7,,227.732929,5.744165,4,1.455682,38962.937500
8,,227.697224,5.712998,5,3.810282,39731.039062
9,,227.655664,5.803919,6,4.874136,39110.574219
10,PSZ2 G006.76+30.45,243.943626,-6.143031,1,0.874235,1673.965942


In [86]:
t2 = Table.from_pandas(df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'INDEX', 'Sep', 'EXPTIME']][:25])

In [87]:
t2.write('table_test.tex', format='latex')

In [89]:
df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'INDEX', 'Sep', 'EXPTIME']][:50].to_latex('table_test.tex', index=False, float_format='%0.2f') 